In [1]:
import numpy as np
import pandas as pd
import random

Read the data

In [2]:
df = pd.read_json("https://raw.githubusercontent.com/acycliq/issplus/master/dashboard/data/img/default/json/iss.json")
GeneExp = np.load('data_preprocessed\GeneExp.npy')
ctc = np.load('data_preprocessed\cell_to_class_map.npy')

Find the best (ie most likely) class for each cell

In [3]:
class_name = df['ClassName']
prob = df['Prob']
best_class = [class_name[n][np.argmax(prob[n])] for n in range(class_name.shape[0])]

Remove class Zero from best_class

In [4]:
best_class = [y for y in best_class if y != 'Zero']

In [5]:
c = random.choice(best_class)
c

'PC.CA1.3'

In [7]:
pd.DataFrame(GeneExp, columns = ctc)

,PC.CA1.1,PC.CA1.1,PC.CA1.1,PC.CA1.1,PC.CA1.1,PC.CA1.1,PC.CA1.1,PC.CA1.1,PC.CA1.1,PC.CA1.1,...,Sst.Nos1,Sst.Nos1,Sst.Nos1,Sst.Nos1,Sst.Nos1,Sst.Nos1,Sst.Nos1,Sst.Nos1,Sst.Nos1,Sst.Nos1
0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,3.0,0.0,0.0,2.0,1.0,2.0,1.0,0.0,...,21.0,15.0,20.0,35.0,9.0,24.0,29.0,31.0,41.0,31.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,3.0,0.0,1.0,0.0,2.0,0.0,1.0,2.0,1.0,...,2.0,2.0,1.0,4.0,2.0,2.0,0.0,4.0,2.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0,4.0,0.0
9,4.0,4.0,5.0,3.0,2.0,0.0,6.0,1.0,7.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
